# DataCosmos API tutorial

Welcome to the DataCosmos API tutorial, in this notebook you will find code examples to

*   get access token to DataCosmos,
*   perform a STAC search on DataCosmos catalougue,
*   access and display a Satellite image
*   perform a band algebra operation



### Before you begin
Make sure you have an environment to run this notebook with installed the required modules:

In [ ]:
pip install mercury folium rasterio

In [ ]:
import json requests
import mercury as mr

## Import the credentials and generate an Auth0 access token
When using any of the DataCosmos APIs you will need to set a bearer token in your HTTP request headers. This bearer token can be retrieved by sending a request to the auth endpoint with your client credentials.

An example of how to use the client credentials to authenticate and receive a bearer token is provided below in different programming languages.

For these examples you will need to save your credentials to `data_cosmos_api_credentials.json` in the same directory using the following format:

```
{
  "client_id": "joULxX1htnXBHt7Q8rqBL3TTICmcMjZ8",
  "client_secret": "<please-put-here-your-client_secret>",
  "audience": "https://beeapp.open-cosmos.com",
  "grant_type": "client_credentials"
}
```



Run the following cell to open the file with the credentials  

In [ ]:
import json

#with open("data_cosmos_api_credentials.json") as fp:
#    client_credentials = json.load(fp)

If instead you run the code in Google Colaboratory it is enough load the shared credentials.

In [ ]:
from google.colab import userdata

client_credentials =  {
                        "client_id": userdata.get('client_id'),
                        "client_secret": userdata.get('client_secret'),
                        "audience": "https://beeapp.open-cosmos.com",
                        "grant_type": "client_credentials"
                      }

Run the next cell in order to get the authorization token


In [ ]:
import requests

session = requests.Session()
req = session.post(
    "https://login.open-cosmos.com/oauth/token",
    data=client_credentials
).json()

session.headers.update(
    {
        "Authorization": f'{req["token_type"]} {req["access_token"]}',
        "Content-Type": "application/json"
    }
)

## Browse DataCosmos STAC catalog

In our catalog we use the STAC standard, which you can learn more about here:

*   [stacspec.org](https://stacspec.org/en)

And also other documentation here:
* [github.com / radiantearth / stac-api-spec](https://github.com/radiantearth/stac-api-spec)
* [github.com / radiantearth / stac-spec](https://)


In particular, this definition will be useful where you can find a definition for a STAC Item:

[github.com / radiantearth / stac-spec / blob / master / item-spec / item-spec.md](https://github.com/radiantearth/stac-spec/blob/master/item-spec/item-spec.md)

To browse the STAC Items in our catalog, please do a `POST` Request to the following endpoint:

`https://app.open-cosmos.com/api/data/v0/scenario/scenario/{project_id}/search`

Where the `project_id` can be any of the following three:

1.   `8b56515d-6d9d-441d-a461-886c98a5e868`
2.   `fd3abdb7-25a1-4b54-ada1-064a9c5bffaa`
3.   `8b56515d-6d9d-441d-a461-886c98a5e868`

Authorization type has to be set to Bearer Token with the token you generated.

An example body for the request would be:


```
{
    "limit": 99,
    "bbox": [-180, -90, 180, 90],
    "query": {}
}
```
where


*  `limit` is the maximum number of items to be returned from the search
*  `bbox` is  the bounding box expressed in coordinates where to search, `[-180, -90, 180, 90]` is the whole earth surface
* `query` is any other possible filter criteria





This will give you a response with different fields: `context`, `features`, `links`, `numberMatched`, `numberReturned` and `type`.
Inside the `features` field, you can find a list with different STAC Items.
Inside every STAC Item there is an `assets` field where you can find the link to access our images.

In [ ]:
import mercury as mr

project_id = userdata.get('project_1')

base_url = "https://app.open-cosmos.com"
url_search = f"{base_url}/api/data/v0/scenario/scenario/{project_id}/search"

params = {
    "limit": 99,
    "bbox": [-180, -90, 180, 90],
    "query": {}
}

collection_response = session.post(url_search, json=params).json()

mr.JSON(collection_response)

# You can print if you cannot see the urls
# print(collection_response)

## Access one item and display the preview

In [ ]:
from IPython.display import Image, display

# URL of the image
image_url = collection_response['features'][2]['assets']['thumbnail']['href']

# Fetch the image
response = session.get(image_url)

# Open the image and display it
image = Image(response.content)

display(image)

In [ ]:
# Fetch the full resolution of the True Colour Image
image_url = collection_response['features'][2]['assets']['TCI']['href']

response = session.get(image_url)

In [ ]:
display(response.content)